In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision import datasets

import matplotlib.pyplot as plt
import os
from PIL import Image
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
path='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Animals'

In [3]:
train_transforms=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_transforms=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])

In [4]:
train_data=datasets.ImageFolder(os.path.join(path,'train'),transform=train_transforms)
test_data=datasets.ImageFolder(os.path.join(path,'test'),transform=test_transforms)

len(train_data),len(test_data)

(12562, 2149)

In [5]:
class_names=train_data.classes
class_dict=train_data.class_to_idx

class_names,class_dict

(['Bird', 'Cat', 'Dog', 'Rabbit'],
 {'Bird': 0, 'Cat': 1, 'Dog': 2, 'Rabbit': 3})

In [6]:
torch.manual_seed(101)
train_loader=DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=32,shuffle=False)

In [7]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer):
    model.train()
    train_loss,train_acc=0,0
    for batch,(X,y) in enumerate(dataloader):
        y_pred=model(X)
        loss=loss_fn(y_pred,y)
        train_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
        train_acc+=(y_pred_class==y).sum().item()/len(y_pred)
    
    train_loss=train_loss/len(dataloader)
    train_acc=train_acc/len(dataloader)
    return train_loss,train_acc

In [8]:
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module):
    model.eval()
    test_loss,test_acc=0,0
    with torch.inference_mode():
        for batch,(X,y) in enumerate(dataloader):
            test_pred_logits=model(X)
            loss=loss_fn(test_pred_logits,y)
            test_loss+=loss.item()
            test_pred_labels=test_pred_logits.argmax(dim=1)
            test_acc+=(((test_pred_labels==y).sum().item())/len(test_pred_labels))
    test_loss=test_loss/len(dataloader)
    test_acc=test_acc/len(dataloader)
    return test_loss,test_acc

In [9]:
from tqdm import tqdm
def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int):
    
    results={'train_loss':[],
             'train_acc':[],
             'test_loss':[],
             'test_acc':[]}
    
    for i in tqdm(range(epochs)):
        train_loss,train_acc=train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer)
        test_loss,test_acc=test_step(model=model,
                                     dataloader=test_dataloader,
                                     loss_fn=loss_fn)
        print(f'Epoch {i} | Train loss: {train_loss:.4f},Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f},Test acc: {test_acc:.4f}')
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    return results

In [10]:
class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(3,32,3,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(32,64,3,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(64,128,3,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(),
            nn.Linear(26*26*128,512),
            nn.ReLU(),
            nn.Linear(512,120),
            nn.ReLU(),
            nn.Linear(120,len(class_names))
        )

    def forward(self,X):
        return F.log_softmax(self.conv(X))

In [11]:
torch.manual_seed(101)
model=Model()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [19]:
torch.manual_seed(42)

n=10
model_results=train(
    model=model,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

 10%|█         | 1/10 [07:28<1:07:19, 448.87s/it]

Epoch 0 | Train loss: 1.0988,Train acc: 0.4636 | Test loss: 1.0449,Test acc: 0.5423


 20%|██        | 2/10 [15:10<1:00:50, 456.31s/it]

Epoch 1 | Train loss: 1.0171,Train acc: 0.5155 | Test loss: 0.9313,Test acc: 0.5832


 30%|███       | 3/10 [22:42<53:01, 454.51s/it]  

...

Epoch 7 | Train loss: 0.5658,Train acc: 0.7735 | Test loss: 0.6908,Test acc: 0.7261


 90%|█████████ | 9/10 [1:08:27<07:43, 463.92s/it]

Epoch 8 | Train loss: 0.5192,Train acc: 0.7910 | Test loss: 0.7502,Test acc: 0.7068


100%|██████████| 10/10 [1:16:39<00:00, 459.95s/it]

Epoch 9 | Train loss: 0.4643,Train acc: 0.8131 | Test loss: 0.6905,Test acc: 0.7537


In [20]:
index=list(range(2149))

c=0
for i in index:
    model.eval()
    with torch.no_grad():
        pred=model(test_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

Number of correct: 1640/2149 | Accuracy: 76.31456491391344


In [3]:
from torchvision import models

In [2]:
model2=models.resnet34(pretrained=True)

/home/amirtesh/Downloads/Computers/Python/PY_ENV/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/amirtesh/Downloads/Computers/Python/PY_ENV/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /home/amirtesh/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [02:34<00:00, 566kB/s]


In [12]:
torch.manual_seed(101)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model2.parameters(),lr=0.001)

In [13]:
model2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
model2.fc=None

In [15]:
model2.fc=nn.Linear(512,len(class_names))

In [16]:
torch.manual_seed(42)

n=3
model_results=train(
    model=model2,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

 33%|███▎      | 1/3 [16:54<33:48, 1014.33s/it]

Epoch 0 | Train loss: 0.4912,Train acc: 0.8190 | Test loss: 0.3038,Test acc: 0.8732


 67%|██████▋   | 2/3 [34:17<17:11, 1031.24s/it]

Epoch 1 | Train loss: 0.3292,Train acc: 0.8842 | Test loss: 0.2446,Test acc: 0.8972


100%|██████████| 3/3 [51:08<00:00, 1022.72s/it]

Epoch 2 | Train loss: 0.2834,Train acc: 0.8974 | Test loss: 0.2866,Test acc: 0.8874


In [18]:
index=list(range(2149))

c=0
for i in index:
    model2.eval()
    with torch.no_grad():
        pred=model2(test_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

Number of correct: 1931/2149 | Accuracy: 89.85574685900419


In [4]:
model3=models.mobilenet_v3_large(pretrained=True)

In [5]:
model3.classifier[3]=nn.Linear(model3.classifier[3].in_features,4)

In [6]:
model3

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [17]:
torch.manual_seed(101)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model3.parameters(),lr=0.001)

In [18]:
torch.manual_seed(42)

n=3
model3_results=train(
    model=model3,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

 33%|███▎      | 1/3 [08:09<16:18, 489.07s/it]

Epoch 0 | Train loss: 0.2599,Train acc: 0.9085 | Test loss: 0.2670,Test acc: 0.9198


 67%|██████▋   | 2/3 [16:18<08:09, 489.53s/it]

Epoch 1 | Train loss: 0.1679,Train acc: 0.9422 | Test loss: 0.2438,Test acc: 0.9265


100%|██████████| 3/3 [24:35<00:00, 491.83s/it]

Epoch 2 | Train loss: 0.1312,Train acc: 0.9540 | Test loss: 0.2882,Test acc: 0.9205


In [7]:
index=list(range(2149))

c=0
for i in index:
    model3.eval()
    with torch.no_grad():
        pred=model3(test_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

NameError: name 'test_data' is not defined